# 1.5.10 Feature Matching + Homography to find Objects

In [1]:
import cv2
import numpy as np

In [3]:
qimg = cv2.imread('face.png',0)
timg = cv2.imread('hu.png',0)

sift = cv2.xfeatures2d.SIFT_create()

kp1, des1 = sift.detectAndCompute(qimg,None)
kp2, des2 = sift.detectAndCompute(timg,None)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE,trees=5)
search_params = dict(chechs=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)

good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)
        
MIN_MATCH_COUNT = 10
if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    
    M, mask = cv2.findHomography(src_pts,dst_pts,cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    
    h,w = qimg.shape
    pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    
    timg = cv2.polylines(timg,[np.int32(dst)],True,255,3,cv2.LINE_AA)

else:
    print("발견된 매칭이 부족합니다 - %d%d" % (len(good),MIN_MATCH_COUNT))
    matchesMask = None
    
draw_params = dict(matchColor = (0,255,0),
                  singlePointColor = None,
                  matchesMask=matchesMask,
                  flags=2)
res = None
res = cv2.drawMatches(qimg,kp1,timg,kp2,good,res,**draw_params)

cv2.imshow("Res",res)
cv2.waitKey(0)
cv2.destroyAllWindows()